### Импортирую pandas для извлечения датасетов
Создаю переменную tweets, где будут только позитивные и отрицательные сообщения (без имен пользователя и тд)\
Вывожу хвост из 3 данных, чтобы убедиться, что в переменной все твиты

In [2]:
import pandas as pd

tweets_neg = pd.read_csv('negative.csv', delimiter=';', 
                    names=['none1', 'none2', 'none3', 'text', 'none4', 'none5', 
                           'none6', 'none7', 'none8', 'none9', 'none10', 'none11'])
tweets_pos = pd.read_csv('positive.csv', delimiter=';', 
                    names=['none1', 'none2', 'none3', 'text', 'none4', 'none5', 
                           'none6', 'none7', 'none8', 'none9', 'none10', 'none11'])
tweets = pd.DataFrame()
tweets['neg'], tweets['pos'] = tweets_neg['text'], tweets_pos['text']
tweets.tail(3)

,neg,pos
111920,"Вот и в школу, в говно это идти уже надо(","@_music_lover_13 оооо, еще и Оля) какая прелес..."
111921,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...","@GazetaRu_All Ныне картина того,как у ЕС,сжало..."
111922,Такси везет меня на работу. Раздумываю приплат...,"RT @bazzzilio: @VRSoloviev :)) Я все еще жив, ..."


Загрузка русских стоп-слов с помощью библиотеки nltk

In [3]:
import nltk
from nltk.corpus import stopwords

russian_stopwords = stopwords.words("russian")

Обрабатываю сообщения пользователей с помощью регулярных выражений и строки стоп-слов

1. Из каждого сообщения оставляю только русские слова
2. Делаю из строки массив слов, где разделитель на слова служит любое кол-во пробелов
3. Создаю строку так, чтобы проверяемое слово не было стоп-словом и длина была не меньше 2 и нормализую его

Функция norm_form запоминает все нормализованные слова, чтобы ускорить программу и каждый раз не запускать morpth.parse.
Иначе программа работает в 8-10 раз медленее.

In [24]:
import re
import pymorphy2

norm = {}
morph = pymorphy2.MorphAnalyzer()

def norm_form(word):
    res = norm.get(word)
    if res:
        return res
    res = morph.parse(word)[0].normal_form
    norm[word] = res
    return res


def clean_tweets(tweets):
  tempArr = []
  for tweet in tweets['neg']:
    tweet = re.sub(r'[^а-яА-Я]', ' ', tweet.lower())
    tweet = re.split(r'\s+', tweet)
    tweet = ' '.join([norm_form(word) for word in tweet if len(word) > 2 and word not in russian_stopwords])
    tempArr.append(tweet)
  tweets['negClean'] = pd.Series(tempArr)
  
  tempArr = []  
  for tweet in tweets['pos']:
    tweet = re.sub(r'[^а-яА-Я]', ' ', tweet.lower())
    tweet = tweet.split()
    tweet = ' '.join([norm_form(word) for word in tweet if len(word) > 2 and word not in russian_stopwords])
    tempArr.append(tweet) 
  tweets['posClean'] = pd.Series(tempArr)

  return tweets
    

tweets = clean_tweets(tweets)
tweets.head(10)

,neg,pos,negClean,posClean
0,на работе был полный пиддес :| и так каждое за...,"@first_timee хоть я и школота, но поверь, у на...",работа полный пиддес каждый закрытие месяц сви...,школотый поверь самый общество профилировать п...
1,"Коллеги сидят рубятся в Urban terror, а я из-з...","Да, все-таки он немного похож на него. Но мой ...",коллега сидеть рубиться долбать винд мочь,таки немного похожий мальчик равно
2,@elina_4post как говорят обещаного три года жд...,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,говорить обещаной год ждать,идиотка испугаться
3,"Желаю хорошего полёта и удачной посадки,я буду...","RT @digger2912: ""Кто то в углу сидит и погибае...",желать хороший пол удачный посадка быть очень ...,угол сидеть погибать голод порция взять хотя ж...
4,"Обновил за каким-то лешим surf, теперь не рабо...",@irina_dyshkant Вот что значит страшилка :D\nН...,обновить какой леший работать простоплеер,значит страшилка блин посмотреть часть создать...
5,"Котёнка вчера носик разбила, плакала и расстра...",ну любишь или нет? — Я не знаю кто ты бля:D ht...,кот нка вчера носик разбить плакать расстраива...,любить знать бля
6,"@juliamayko @O_nika55 @and_Possum Зашли, а то ...","RT @SpoonLamer: Ох,900 :D ну это конечно же @t...",заслать затихариться прямо физически страдать ...,это друг такой мимими
7,а вообще я не болею - я не выздоравливаю :(,RT @veregijytaqo: У тебя есть ухажёр? Нет - мо...,вообще болеть выздоравливать,ухаж мой ухо
8,я микрофраза :( учимся срать кирпичами в режим...,Поприветствуем моего нового читателя @Alexey17...,микрофраза учиться срать кирпич режим нона стоп,поприветствовать мой новый читатель
9,"я хочу с тобой помириться , но сука я гордая и...",Теперь у меня есть частичка Сиднея :) #Sydney ...,хотеть ты помириться сука гордый сделать,частичка сидней


In [23]:
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer

cv = CountVectorizer()
count_vector=cv.fit_transform(tweets['neg'])